In [ ]:
# signature plalist endpoint 
def md5hex(s: str) -> str:
    md5_bin = hashlib.md5(s).digest()
    return base64.urlsafe_b64encode(md5_bin).rstrip(b"=").decode("ascii")

def sign_uri(path_uri: str, expires_ts: int) -> str:
    # signature computed from: <st> + <uri> + <SECRET>
    raw = f"{expires_ts}{path_uri}{URI_SIGNATURE_SECRET}".encode("utf-8")
    return md5hex(raw)



@router.get("/playlist/{video_id}/{resolution_path}")
def signed_segments(video_id: str, resolution_path: str):
    # fetch original master playlist from MinIO
    try:
        s3_key = f"videos/{video_id}/{resolution_path}/playlist.m3u8"
        response = s3.get_object(Bucket=STREAMING_BUCKET, Key=s3_key)
        playlist_content = response['Body'].read().decode('utf-8')
    except Exception as e:
        raise HTTPException(404, f"Failed to fetch master playlist: {str(e)}")

    original = playlist_content.splitlines()
    expires = int(time.time()) + TTL

    def make_modified_line(line: str) -> str:
        # if line is a comment or empty, return unchanged
        if line.strip() == "" or line.startswith("#"):
            return line
        
        # lines are like eg: "seg_001.ts"
        
        
        public_path = f"/videos/{video_id}/{resolution_path}/{line.lstrip('/').rstrip('/')}"

        sig = sign_uri(public_path, expires)
        return f"{public_path}?st={expires}&sig={sig}"

    rewritten_lines = [make_modified_line(line) for line in original]
    rewritten_playlist = "\n".join(rewritten_lines) + "\n"
    return Response(content=rewritten_playlist, media_type="application/vnd.apple.mpegurl")

In [ ]:
location ~ ^/videos/(?<video_dir>[^/]+/[^/]+)/(.*)$ {
            # --- CORS Configuration for Video Player ---
            # Handles browser "preflight" permission checks.
            if ($request_method = 'OPTIONS') {
                add_header 'Access-Control-Allow-Origin' 'http://localhost' always;
                add_header 'Access-Control-Allow-Methods' 'GET, OPTIONS' always;
                add_header 'Access-Control-Allow-Headers' 'Range, Content-Type' always;
                add_header 'Access-Control-Max-Age' 1728000;
                return 204;
            }
            # Adds the permission header to the actual video file responses.
            add_header 'Access-Control-Allow-Origin' 'http://localhost' always;
            # --- End CORS Configuration ---

            # secure_link reads signature from query arg 'sig'
            secure_link $arg_sig,$arg_st;
            # compute expected md5 as md5("<st><uri><secret>")
            secure_link_md5 "$secure_link_expires/videos/$video_dir/$secure_link_secret";

            # Auth and error handling
            if ($secure_link = "") {
                return 403;
            }
            if ($secure_link = "0") {
                return 410;
            }

            # caching 
            proxy_cache hls_cache;
            proxy_cache_key "$scheme$host$uri";
            proxy_cache_valid 200 206 10m;
            proxy_cache_lock on;
            proxy_cache_use_stale error timeout updating http_500 http_502 http_503 http_504;
            
            
            # Streaming optimizations
            proxy_set_header Connection '';
            proxy_buffering off;
            proxy_read_timeout 3600s;
            
            # Proxy to MinIO
            proxy_pass http://127.0.0.1:9000/streaming/videos/$video_dir/$2;

            add_header X-Cache-Status $upstream_cache_status;
            proxy_set_header Host $host;
        }

In [ ]:
import redis.asyncio as redis

@router.get("/playlist/{video_id}/{resolution_path}")
async def signed_segments(video_id: str, resolution_path: str, request: Request):

    s3_client = request.app.state.s3_client

    # fetch original master playlist from MinIO
    try:
        s3_key = f"videos/{video_id}/{resolution_path}/playlist.m3u8"
        response = await s3_client.get_object(Bucket=STREAMING_BUCKET, Key=s3_key)
        playlist_bytes = await response['Body'].read()
        playlist_content = playlist_bytes.decode('utf-8')
    except Exception as e:
        raise HTTPException(404, f"Failed to fetch master playlist: {str(e)}")

    # original = playlist_content.splitlines()
    expires = int(time.time()) + TTL
    # This is our helper function that re.sub will call for each segment line
    def replacer(match):
        # The matched line (e.g., "seg_001.ts") is the first captured group
        segment_file = match.group(1)
        
        public_path = f"/videos/{video_id}/{resolution_path}/{segment_file}"
        sig = sign_uri(public_path, expires)
        
        return f"{public_path}?st={expires}&sig={sig}"

    # This regex finds any line that does NOT start with '#' or whitespace,
    # and captures the entire line. re.MULTILINE makes '^' match the start of each line.
    pattern = re.compile(r"^([^#\s].*)$", re.MULTILINE)
    
    # Use re.sub to perform the replacement in a single, fast operation
    rewritten_playlist = pattern.sub(replacer, playlist_content)
    
    return Response(content=rewritten_playlist, media_type="application/vnd.apple.mpegurl")


In [2]:
import os
print(os.cpu_count())

12


In [1]:
import platform

platform.python_implementation()

'CPython'